# 背景

本项目是记录在参加百度顶会深度论文复现营时的实操经历和分享论文复现过程中一些浅薄的经验。百度资深算法工程师与中科院高级研究员联合授课，28天手把手带领我们掌握论文复现全流程。

课程链接：[百度顶会深度论文复现营](https://aistudio.baidu.com/aistudio/education/group/info/1340)


复现论文： ECO Online Video Understanding

链接： https://arxiv.org/pdf/1804.09066.pdf

pytorch代码链接：  https://github.com/mzolfaghari/ECO-pytorch

我的论文阅读笔记： https://zhuanlan.zhihu.com/p/169286560



# 论文简单阅读总结

1. **这篇论文主要立足两大视频领域的问题:** 

大部分推理均是局部进行的，丢失了跨越数秒的动作内的关联关系, 尽管有快速桢处理的局部方法，整个视频的处理效率不高，有损在线长视频分类检索的效率

2. **最大的特点**：

相比之前的3d结构网络, 不但计算复杂，且小窗口无法覆盖整个视频。现存方法典型做法是应用基于窗口分数的特定后融合，探索窗口间的时序关系并不是最优的。个人认为这很显著的解决了工程上的问题, 比如很难真正的对视频进行实时分析.

3. **本篇论文的主要贡献：**

作者提出了端到端的训练结构. 对相邻帧的大部分冗余信息也作了很好的处理, 对跨度比较大的长远帧的上下文信息也很好的补货, 极大提高仅从单帧获得的置信度belief，尤其是对复杂的长范围动作。

4. **网络结构拓扑图：**

![](https://ai-studio-static-online.cdn.bcebos.com/2680edae24824365b5456846504cf7031fdcf2d8207b4b35844db116763559b1)


5. **视频帧采样方式**

将视频均分成N个子集subsections, 每个subsection只采样一阵, 并将这些帧累计成到N个, 然后交给3d cnn处理, 以这种方式考虑时序信息, 完成动作的分类.

6. **数据增强：**

跟TSN非常类似. 输入204320, 并使用角裁剪和尺度抖动,加上翻转等方式, 最终中心裁剪到224*224的大小送入网络.

本文提出了一个简单且高效的网络体系结构，仅查看的a subset of frames from a video，学习这些帧间的时间上下文。这个原理可用于各种视频理解任务。动作分类，在线动作分类，视频字幕等表现出色。计算负载和内存占用使得在移动设备的实现成为未来一可行选项，这些方法比SOTA快10-80倍。

**优点：**

* ECO也采用了3D网络, 对于长视频也作了稀疏采样策略, 较好的捕获了长期信息
* ECO是端到端的建模方法，不需要提取光流或RGB
* ECO最显著的地方就是快, 方便工程实践使用, 同时准确率也还可以
* ECO的网络结果, 直观的motivation+简单有效的框架设计

**不足：**

* 准确率上还差了那么一丢丢
* 没有使用音频等信息,用于多模态学习


# 论文复现主要过程

复现过程主要基于pytorch源码进行研究分析，对比pytorch和paddlepaddle的API， 课上老师也给了二者详细的API对照列表，个人认为仔细阅读API文档非常重要。

参考： [PaddlePaddle API在线文档](https://www.paddlepaddle.org.cn/documentation/docs/zh/api_guides/index_cn.html)。

复现过程中也使用paddle的工具x2paddle， 将开源的onnx直接转为pdmodel， 对于查看网络结构有很好的参考作用。

对于网络结构的搭建，个人建议使用动态图的方式，非常方便，打印网络信息和网络灵活的构造，分模块搭建，对输入进行模拟测试。

老师已将论文复现的方法轮总结如下，非常受益：

![](https://ai-studio-static-online.cdn.bcebos.com/25fa6c3ec52647f39f6496f688e0cd860556222bd88a49b5a9bc81d01f96eb85)


### （1）参数配置

In [ ]:
!cd /home/aistudio/work/ECO-paddle && python test_config.py "ucf101" RGB train.txt val.txt

------------------------------------
resnet101 Configurations:
- dataset: ucf101
- modality: RGB
- train_list: train.txt
- val_list: val.txt
- net_model: None
- net_model2D: None
- net_modelECO: None
- net_model3D: None
- arch: resnet101
- num_segments: 3
- consensus_type: avg
- pretrained_parts: both
- k: 3
- dropout: 0.5
- loss_type: nll
- epochs: 45
- batch_size: 256
- iter_size: 1
- lr: 0.001
- lr_steps: [20, 40]
- momentum: 0.9
- weight_decay: 0.0005
- clip_gradient: None
- no_partialbn: False
- nesterov: False
- num_saturate: 5
- print_freq: 20
- eval_freq: 5
- workers: 4
- resume: 
- evaluate: False
- snapshot_pref: 
- start_epoch: 0
- gpus: None
- flow_prefix: 
- rgb_prefix: 
------------------------------------


### （2）数据预处理
1. 解压数据集并生成list
2. 对数据进行预处理

问题： UCF拆成图片实在太大了，所以还是用tsn的示例数据hmdb_data_demo

**预处理代码逻辑分析如下：**

1. video_loader函数会变化，采样规则不一样，没有seglen
2. image加载方式相同，都是PIL的RGB

	- 注意训练时有random_shift, 而测试时没有
3. 数据增强：

    rgb训练:  GroupMultiScaleCrop(self.input_size, [1, .875, .75, .66], GroupRandomHorizontalFlip(is_flow=False)])
    
   	 rgb测试:  GroupScale(int(scale_size)), GroupCenterCrop(crop_size),
4. Stack(roll=True)自定义函数，在transform：， 就是多个图像的合并操作
5. ToTorchFormatTensor， HWC to CHW， 转格式， 但是貌似并没有除255, 注意mean和std不一样：
            self.input_size = 224
            self.input_mean = [104, 117, 128]
            self.input_std = [1]
6. normalize， GroupNormalize(input_mean, input_std)， 均值归一化，值都一样


In [ ]:
# !unzip /home/aistudio/data/data48916/UCF-101.zip  -d /home/aistudio/work/ucf101

#对ucf生成光流数据, 太慢了
#!python work/create_flow.py

#尝试直接生成jpg数据
# !python work/create_rgb.py

#生成list, list占用自带的
#!python work/ECO-pytorch/gen_dataset_lists.py ucf101 /home/aistudio/work/ucf101/frames #原pytorch代码已经生成好


#使用hmdb_data_demo
# !tar xf data/data16432/hmdb_data_demo.tar -C work/ 2>/dev/null
# !python work/ECO-paddle/avi2jpg.py
# !python work/ECO-paddle/jpg2pkl.py
# !python work/ECO-paddle/data_list_gender.py

#使用UCF-101
# !python /home/aistudio/work/ECO-paddle/avi2jpg.py
# import os
# import shutil
# dirs = os.listdir('/home/aistudio/work/ucf101/UCF-101') #删除视频文件，腾出
# for d in dirs:
#     if not d.endswith("_jpg"):
#         shutil.rmtree( '/home/aistudio/work/ucf101/UCF-101/' + d )
# 空间太大，使用大佬已经生成帧的数据集UCF-101-jpg, 并命令行使用解压命令  tar -zxvf /home/aistudio/data/data49167/UCF-101-jpg.tgz
# !python work/ECO-paddle/avi2jpg.py #生成label
# !python work/ECO-paddle/ucf_jpg2pkl.py #生成pkl
!python work/ECO-paddle/data_list_gender.py

10627
1296
1397


In [ ]:
#测试reader
%cd /home/aistudio/work/ECO-paddle/
from hmdb_reader import KineticsReader
from cfg_config import parse_config, merge_configs, print_configs
config = parse_config('/home/aistudio/work/ECO-paddle/configs/hmdb_config_5.txt')
# print(config)
train_config = merge_configs(config, 'test', {})
train_reader = KineticsReader('tsn', 'test',train_config ).create_reader()
for batch_id, data in enumerate(train_reader()):
    print(data.shape)
    break

/home/aistudio/work/ECO-paddle
{'MODEL': {'name': 'eco_full', 'format': 'pkl', 'num_classes': 101, 'seg_num': 32, 'seglen': 1, 'image_mean': [104, 117, 128], 'image_std': [1, 1, 1], 'num_layers': 50, 'new_length': 1, 'modality': 'RGB'}, 'TRAIN': {'epoch': 45, 'short_size': 240, 'target_size': 224, 'num_reader_threads': 1, 'buf_size': 1024, 'batch_size': 16, 'use_gpu': True, 'num_gpus': 1, 'filelist': '/home/aistudio/work/UCF-101-jpg/train.list', 'learning_rate': 0.003, 'learning_rate_decay': 0.1, 'l2_weight_decay': 0.0001, 'momentum': 0.9, 'total_videos': 80}, 'VALID': {'short_size': 240, 'target_size': 224, 'num_reader_threads': 1, 'buf_size': 1024, 'batch_size': 2, 'filelist': '/home/aistudio/work/UCF-101-jpg/val.list'}, 'TEST': {'seg_num': 32, 'short_size': 240, 'target_size': 224, 'num_reader_threads': 1, 'buf_size': 1024, 'batch_size': 10, 'filelist': '/home/aistudio/work/UCF-101-jpg/test.list'}, 'INFER': {'short_size': 240, 'target_size': 224, 'num_reader_threads': 1, 'buf_size':

AttributeError: 'list' object has no attribute 'shape'

### （3）搭建网络
1. 网络结构是ECOfull: https://gitee.com/thunder95/ECO-pytorch/blob/master/tf_model_zoo/ECOfull/ECOfull.yaml
2. 先将模型文件转为onnx, 可用netron在线查看网络结构,参考以下代码，若有报错需要注释掉consensus的output：

```
    ckpt = torch.load('/home/hl/tests/ECO-pytorch/models/eco_lite_rgb_16F_kinetics_v1.pth.tar')
    from torch.autograd import Variable
    model.load_state_dict(ckpt['state_dict'], strict=False)
    model.eval()
    input = torch.randn(1, 16, 3, 224, 224)
    torch.onnx.export(model, input, "emotion.onnx", verbose=True)
    print("==>模型转换成功！")
```


3. x2paddle可以直接转换onnx到paddle模型和代码，作为很好的参考
	* 在转换过程中，x2paddle因pad报错， 暂放弃， 见https://github.com/PaddlePaddle/X2Paddle/issues/375
	* 已生成的部分也可以作为参考：work/ECO-paddle/onnx/pd_models/model_with_code/model.py
4. 输入shape: batch_size * num_segments * 224 * 224
5. 手动搭建网络， 参考TSN和inception网络结构

In [ ]:
#!pip install x2paddle onnx==1.6.0 sympy packaging
# !x2paddle --framework=onnx --model=/home/aistudio/work/ECO-paddle/onnx/kinetics_16.onnx --save_dir=/home/aistudio/work/ECO-paddle/onnx/pd_models


###  （4）开始训练

In [3]:
%cd /home/aistudio/work/ECO-paddle/
!python train.py --use_gpu True --config /home/aistudio/work/ECO-paddle/configs/hmdb_config_5.txt --epoch 100 --pretrain True

/home/aistudio/work/ECO-paddle
W0907 08:40:09.761824   314 device_context.cc:252] Please NOTE: device: 0, CUDA Capability: 70, Driver API Version: 9.2, Runtime API Version: 9.0
W0907 08:40:09.766607   314 device_context.cc:260] device: 0, cuDNN Version: 7.3.
Loss at epoch 0 step 0: [0.50458], acc: [0.9]
Loss at epoch 0 step 50: [0.5282265], acc: [0.9]
Loss at epoch 0 step 100: [0.65667844], acc: [0.9]
Loss at epoch 0 step 150: [0.14322658], acc: [1.]
Loss at epoch 0 step 200: [0.64588934], acc: [0.8]
Loss at epoch 0 step 250: [0.33731565], acc: [0.9]
Loss at epoch 0 step 300: [0.72403073], acc: [0.8]
Loss at epoch 0 step 350: [1.0441022], acc: [0.7]
Loss at epoch 0 step 400: [0.8526255], acc: [0.9]
Loss at epoch 0 step 450: [1.0301446], acc: [0.7]
Loss at epoch 0 step 500: [0.8235967], acc: [0.7]
Loss at epoch 0 step 550: [0.6799974], acc: [0.8]
Loss at epoch 0 step 600: [0.48879313], acc: [0.9]
Loss at epoch 0 step 650: [0.889543], acc: [0.9]
Loss at epoch 0 step 700: [0.15427744], ac

### （5）测试集效果

In [6]:
%cd /home/aistudio/work/ECO-paddle/
!python test.py --use_gpu True --config /home/aistudio/work/ECO-paddle/configs/hmdb_config_4.txt --weights /home/aistudio/work/ECO-paddle/checkpoints_models/best_model_saved.pdparams

/home/aistudio/work/ECO-paddle
W0907 10:07:00.039813  3546 device_context.cc:252] Please NOTE: device: 0, CUDA Capability: 70, Driver API Version: 9.2, Runtime API Version: 9.0
W0907 10:07:00.044451  3546 device_context.cc:260] device: 0, cuDNN Version: 7.3.
验证集准确率为:0.9158950448036194
损失为:0.35290399193763733


# 总结

这次参加的论文复现营非常有意义，以前都是伸手党的形式无脑的进行模型训练，并未深入思考过模型网络结构，更没有老师手把手教我的对网络结构深层次分析。虽收获良多，但本次项目发现个人的很多不足，比如数据和精度对齐，模型的应用实践都还没做，后续有时间还需深入学习。

欢迎各位大佬一起探讨学习，求AI关注互粉：https://aistudio.baidu.com/aistudio/personalcenter/thirdview/89442

**非常感谢各位老师, 助教, PaddlePaddle团队的倾情奉献！**



请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 